In [1]:
import re
import multiprocessing
import os


from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [2]:
class RawQuery(Dataset):
    def __init__(self, raw_path='data'): 
        # read provided data
        raw_training_data = pd.read_csv(os.path.join(raw_path, 'TD.csv'))
        news_urls = pd.read_csv(os.path.join(raw_path, 'NC_1.csv'))
        contents = pd.read_json(os.path.join(raw_path, 'url2content.json'), typ=pd.Series)
        
        # proccess data
        merged_training = pd.merge(raw_training_data, news_urls, on=['News_Index'], how='left')
        contents = contents.apply(lambda x: ' [CLS] ' + re.sub('[\n，。、]+', ' [SEP] ', x))
        
        # construct dataset
        self.queries = merged_training['Query']
        self.contents = contents[merged_training['News_URL']]
        self.targets = merged_training['Relevance']
        
        self.size = len(merged_training)
        
    def __getitem__(self, i):
        
        padding_size = 512
        tokenized_text = tokenizer.tokenize(self.contents[i])

        if len(tokenized_text) > padding_size:
            tokenized_text = tokenized_text[:padding_size]
        else:
            tokenized_text = tokenized_text + ['[PAD]'] * (padding_size - len(tokenized_text))

        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

        segments_tensors = torch.zeros(padding_size).long()
        tokens_tensor = torch.tensor(indexed_tokens)
        
        return self.queries[i], tokens_tensor, segments_tensors, self.targets[i]
    
    def __len__(self):
        return self.size


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
bert = BertModel.from_pretrained('bert-base-chinese')
bert = bert.cuda()
bert.eval();


NumberCPU = multiprocessing.cpu_count()


In [5]:
dataset = RawQuery()
loader = DataLoader(
    dataset, batch_size=70, shuffle=False, num_workers=NumberCPU
)

encoded_text = []
with torch.no_grad():
    for q, t, s, y in loader:
        _, encoded_batch = bert(t.cuda(), s.cuda())
        encoded_text.append(encoded_batch.cpu().data.numpy())

np.save('data/embedded/encoded_content.npy', np.concatenate(encoded_text, axis=0))

In [7]:
raw_training_data = dataset.queries

queries = list(set(raw_training_data))
encoded_text = []
with torch.no_grad():
    for q in queries:
        tokenized_text = tokenizer.tokenize(q)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_tensors = torch.zeros(len(indexed_tokens)).long().cuda()
        tokens_tensor = torch.tensor([indexed_tokens]).cuda()
        _, encoded = bert(tokens_tensor, segments_tensors)
        encoded_text.append(encoded.cpu().data.numpy())

df = pd.DataFrame()
df['Query'] = queries
df['code'] = encoded_text
encoded_query = pd.merge(raw_training_data, df, on=['Query'], how='left')['code']
np.save('data/embedded/encoded_query.npy', np.concatenate(encoded_query, axis=0))

In [12]:
raw_training_data = dataset.queries

queries = pd.unique(raw_training_data)
encoded_text = []
with torch.no_grad():
    for q in queries:
        tokenized_text = tokenizer.tokenize(q)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_tensors = torch.zeros(len(indexed_tokens)).long().cuda()
        tokens_tensor = torch.tensor([indexed_tokens]).cuda()
        _, encoded = bert(tokens_tensor, segments_tensors)
        encoded_text.append(encoded.cpu().data.numpy())

np.save('data/embedded/ref_training_uniq_query.npy', queries)
np.save('data/embedded/encoded_training_uniq_query.npy', np.concatenate(encoded_text, axis=0))

In [8]:
class NewsContentSet(Dataset):
    def __init__(self, raw_path='data'): 
        # read provided data
        news_urls = pd.read_csv(os.path.join(raw_path, 'NC_1.csv'))
        contents = pd.read_json(os.path.join(raw_path, 'url2content.json'), typ=pd.Series)
        
        # proccess data
        contents = contents[news_urls['News_URL']]
        contents = contents.apply(lambda x: ' [CLS] ' + re.sub('[\n，。、]+', ' [SEP] ', x))
        
        # construct dataset
        self.contents = contents        
        self.size = len(contents)
        
    def __getitem__(self, i):
        padding_size = 512
        tokenized_text = tokenizer.tokenize(self.contents[i])

        if len(tokenized_text) > padding_size:
            tokenized_text = tokenized_text[:padding_size]
        else:
            tokenized_text = tokenized_text + ['[PAD]'] * (padding_size - len(tokenized_text))

        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

        segments_tensors = torch.zeros(padding_size).long()
        tokens_tensor = torch.tensor(indexed_tokens)
        
        return tokens_tensor, segments_tensors
    
    def __len__(self):
        return self.size
    

In [12]:
queries = pd.read_csv(os.path.join('data', 'QS_1.csv'))['Query']
encoded_text = []
with torch.no_grad():
    for q in queries:
        tokenized_text = tokenizer.tokenize(q)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_tensors = torch.zeros(len(indexed_tokens)).long().cuda()
        tokens_tensor = torch.tensor([indexed_tokens]).cuda()
        _, encoded = bert(tokens_tensor, segments_tensors)
        encoded_text.append(encoded.cpu().data.numpy())


np.save('data/embedded/encoded_test_query.npy', np.concatenate(encoded_text, axis=0))

In [17]:
loader = DataLoader(
    NewsContentSet(), batch_size=70, shuffle=False, num_workers=NumberCPU
)

encoded_text = []
with torch.no_grad():
    for t, s in loader:
        _, encoded_batch = bert(t.cuda(), s.cuda())
        encoded_text.append(encoded_batch.cpu().data.numpy())

np.save('data/embedded/encoded_all_content.npy', np.concatenate(encoded_text, axis=0))

In [34]:
for i in range(30):
    print(dataset[i][1][-30:].replace('\n', ''))

同等重視，讓獄政基本人權邁一大步。1031230(中央社)
表示，這支重型狙擊步槍可有效對抗輕型裝甲車。(中時電子報)
一份新的診斷報告，而是放下藍綠、尊重人權的決心。(旺報 )
、台中市廖姓台商及屏東縣前國民黨縣議長等勢力的申請案。()
，並一起向現場冉冉上升的國旗致敬。1040101(中央社)
權威，不會提升台灣國際形象，只會深化政治對立，加重社會傷痕。
程出訪歐洲前，還特別去看過陳水扁。1030124(中央社)
府同意讓健康惡化的陳水扁保外就醫資料來源：《蘋果》採訪整理
樣的做法，否則阿扁情況一定會更糟。1020419(中央社)
縮社會運動陳情抗議空間，對台灣公民社會發展也將造成嚴重阻礙。
期的規定，法務部必須依法行政，目前須讓扁在醫療專區繼續照顧。
明蝦和紅蟳通通都有，地方親友還是滿心祝賀，這對新人百年好合。
支持後，相信馬政府「頑石亦能點頭」，終會同意保外就醫。()
黨漸失人心時，民進黨卻仍無法得利，獲得人民信任與支持。()
立法院，要等到公式過關、台電回饋，遙遙無期。(中時電子報)
，才出面聲援阿扁保外就醫，這些動作純粹是基於人道考量。()
很清楚，馬應立即實踐諾言，讓扁保外就醫，實現醫療人權的承諾。
有當召集人不是那麼重要，現在剩下政治上怎麼解決。(旺報 )
由巷」揭牌儀式時，呼籲讓扁保外就醫資料來源：《蘋果》資料室
郝出來開了第一槍，但他不覺得郝是藍營內部唯一贊成的人。()
法核准保外醫治，讓陳水扁回家療養。1031210(中央社)
這些社團要立委們一個一個用書面表態，「我們能不簽嗎？」()
望集結社會各界的力量，能讓阿扁在過年前，達成保外就醫的願望。
：輕度狹心症、高血脂症★膚色莫名變深資料來源：扁醫療小組
醫」，扁上警備車後，還有支持者跳上車頭阻擋，隨即被警員拉下。
治和諧考量，應由馬總統特赦阿扁，這是處理阿扁問題的最好方法。
並起，南台灣無人要迎戰的的政局，也開始要讓馬英九頭疼。()
及吳榮義明確表達支持赦扁；吳允諾當選成立「救扁小組」。()
告確已收到，至於是否保外就醫，由矯正署會同台中監獄審慎處理。
音，讓世界看見台灣民主和平發展，更尊重人道，重視人權。()
